# Evaluation

In [1]:
save_path = r"\depth_mode\Savings\jp-gcdt-xlm-roberta-base_bs1_seed111\\"
data_path = r"\data\pickle-data\depth\to_pt\jp-gcdt-xlm-roberta-base"

In [2]:
import math
import torch.optim as optim
import random
import torch.nn as nn
import copy
import os
import glob
import Metric
from Metric import getBatchMeasure, getMicroMeasure, getMacroMeasure

In [3]:
import re
import os
import pickle
import torch
import numpy as np
import random
from Training import Train, EvalOnly, getAccuracy
import time
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel, MBart50Tokenizer
import config
from model_depth import ParsingNet

os.environ["CUDA_VISIBLE_DEVICES"] = str(config.global_gpu_id)
base_path = config.tree_infer_mode + "_mode/"


USE_CUDA = torch.cuda.is_available() #False
device = torch.device("cuda:" + str(config.global_gpu_id) if USE_CUDA else "cpu") #device(type='cpu')

In [4]:
batch_size = 1
hidden_size = config.hidden_size
rnn_layers = 1
dropout_e = 0.5
dropout_d = 0.5
dropout_c = 0.5
input_is_word = 'True'
atten_model = 'Dotproduct'
classifier_input_size = config.hidden_size #768
classifier_hidden_size = int(config.hidden_size / 1) #768
classifier_bias = 'True'
use_org_Parseval = 'True'
eval_only = 'True'

seednumber = 111
data_path_split = [x for x in data_path.split(os.sep) if x != ""]
data_base = data_path_split[-1] #'zh-gcdt-hfl-chinese-roberta-wwm-ext'
savepath = r"Savings\jp-gcdt-xlm-roberta-base_bs1_seed111\\"
finetuning = "False"
if savepath == None and finetuning == "True":
    assert False
elif savepath == None and finetuning == "False":
    save_path = base_path + "Savings/%s_bs%d_seed%d/" % (data_base, batch_size, seednumber)
    finetune_frompath = None
elif savepath != None and finetuning == "True":
    finetune_frompath = base_path + savepath
    save_path = finetune_frompath.replace("Savings/", "Savings/finetuned_")
elif savepath != None and finetuning == "False":
    finetune_frompath = None
    save_path = base_path + savepath
        
eval_size = 1
epoch = 50
lr = 0.0002
lr_decay_epoch = 1
weight_decay = 0.01

In [5]:
# get language embedding
language_embedding_name = re.sub(r".*jp-", "", re.sub(r".*gcdt-", "", (re.sub(r".*rstdt-", "", re.sub(r".*gum-", "", data_base)))))
language_embedding_name = language_embedding_name.replace("hfl-", "hfl/").replace("SpanBERT-", "SpanBERT/")
# 'hfl/chinese-roberta-wwm-ext' a model on Hugging Face

""" BERT tokenizer and model """
print("language embedding name: ", language_embedding_name)
bert_tokenizer = AutoTokenizer.from_pretrained(language_embedding_name, use_fast=True)
bert_model = AutoModel.from_pretrained(language_embedding_name)
print("Language embedding loaded from pretrained: ", language_embedding_name)

""" freeze some layers """
for name, param in bert_model.named_parameters():
    layer_num = re.findall("layer\.(\d+)\.", name)
    if len(layer_num) > 0 and int(layer_num[0]) > 2:
        param.requires_grad = True
    else:
        param.requires_grad = False

if USE_CUDA:
    language_model = bert_model.cuda()
else:
    language_model = bert_model

# Setting random seeds 
torch.manual_seed(seednumber)
if USE_CUDA:
    torch.cuda.manual_seed_all(seednumber)
np.random.seed(seednumber)
random.seed(seednumber)

# Process bool args       
if classifier_bias == 'True':
    classifier_bias = True
    # True

elif classifier_bias == 'False':
    classifier_bias = False

language embedding name:  xlm-roberta-base
Language embedding loaded from pretrained:  xlm-roberta-base


In [6]:
Tr_InputSentences = []
Tr_EDUBreaks = []
Tr_DecoderInput = []
Tr_RelationLabel = []
Tr_ParsingBreaks = []
Tr_GoldenMetric = []
Tr_ParentsIndex = []
Tr_SiblingIndex = []

Dev_InputSentences = []
Dev_EDUBreaks = []
Dev_DecoderInput = []
Dev_RelationLabel = []
Dev_ParsingBreaks = []
Dev_GoldenMetric = []
Dev_ParentsIndex = []
Dev_SiblingIndex = []

# Load Testing data
Test_InputSentences = []
Test_EDUBreaks = []
Test_DecoderInput = []
Test_RelationLabel = []
Test_ParsingBreaks = []
Test_GoldenMetric = []

# Load Training data
Tr_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Training_InputSentences.pickle"), "rb")))
Tr_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Training_EDUBreaks.pickle"), "rb")))
Tr_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Training_DecoderInputs.pickle"), "rb")))
Tr_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Training_RelationLabel.pickle"), "rb")))
Tr_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Training_ParsingIndex.pickle"), "rb")))
Tr_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Training_GoldenLabelforMetric.pickle"), "rb")))
Tr_ParentsIndex.extend(pickle.load(open(os.path.join(data_path, "Training_ParentsIndex.pickle"), "rb")))
Tr_SiblingIndex.extend(pickle.load(open(os.path.join(data_path, "Training_Sibling.pickle"), "rb")))

# Load Deving data
Dev_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Deving_InputSentences.pickle"), "rb")))
Dev_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Deving_EDUBreaks.pickle"), "rb")))
Dev_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Deving_DecoderInputs.pickle"), "rb")))
Dev_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Deving_RelationLabel.pickle"), "rb")))
Dev_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Deving_ParsingIndex.pickle"), "rb")))
Dev_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Deving_GoldenLabelforMetric.pickle"), "rb")))
Dev_ParentsIndex.extend(pickle.load(open(os.path.join(data_path, "Deving_ParentsIndex.pickle"), "rb")))
Dev_SiblingIndex.extend(pickle.load(open(os.path.join(data_path, "Deving_Sibling.pickle"), "rb")))

# Load Testing data
Test_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Testing_InputSentences.pickle"), "rb")))
Test_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Testing_EDUBreaks.pickle"), "rb")))
Test_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Testing_DecoderInputs.pickle"), "rb")))
Test_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Testing_RelationLabel.pickle"), "rb")))
Test_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Testing_ParsingIndex.pickle"), "rb")))
Test_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Testing_GoldenLabelforMetric.pickle"), "rb")))

In [7]:
# To check data
sent_temp = ''
print("Checking Data...")
for word_temp in Tr_InputSentences[2]:
    sent_temp = sent_temp + ' ' + word_temp
print(sent_temp)
print('... ...')
print("That's great! No error found!")
print("All train sample number:", len(Tr_InputSentences))

Checking Data...
 ▁ 加 藤 ▁ 周一 ▁ の ▁「 ▁ 眼 ▁ 」 ▁ と ▁「 ▁ 耳 ▁ 」 ▁ 加 國 ▁ 尚 志 ▁ メ ル ロ ▁= ▁ ポン ティ ▁ 晩 年 ▁ の ▁ 哲学 ▁ 概念 ▁ に ▁「 ▁ 肉 ▁la ▁chair ▁ 」 ▁ と ▁ いう ▁ 概念 ▁ が ▁ ある ▁ 。 ▁ これ ▁ は ▁ たとえば ▁ 右手 ▁ と ▁ 左 手 ▁ を ▁ 重 ね 合わせ る ▁ とき ▁ 、 ▁ 触れ る ▁ 手 ▁ と ▁ 触れ ▁ られる ▁ 手 ▁ の ▁ 間 ▁ に ▁ 可 逆 ▁ 的 ▁ で ▁ あい まい ▁ な ▁ 感覚 ▁ が ▁ 生 じ ▁ 、 ▁ 主 観 ▁ と ▁ 客 観 ▁ 、 ▁ 能 動 ▁ と ▁ 受 動 ▁ が ▁ 相互 ▁ に ▁ 移 り ▁ 行き あ う ▁ よう ▁ な ▁「 ▁ 感じ ▁ られる ▁ もの ▁le ▁sensible ▁ 」 ▁ を ▁ 言い 表 し ▁ た ▁ もの ▁ で ▁ ある ▁ 。 ▁この ▁「 ▁ 感じ ▁ られる ▁ もの ▁ 」 ▁ は ▁「 ▁ 見える ▁ もの ▁ 」 ▁ や ▁「 ▁ 触れ ▁ う る ▁ もの ▁ 」 ▁ に ▁ 宿 り ▁ 、 ▁ さらに ▁ は ▁それ ▁ ら ▁ 相互 ▁ の ▁ 転 移 ▁ を ▁ 可能 ▁ に ▁ する ▁ 。 ▁この ▁「 ▁ 肉 ▁ 」 ▁ の ▁ 経験 ▁ が ▁ たとえば ▁ 色 ▁ など ▁ の ▁ 質 ▁ の ▁ 経験 ▁ の ▁ 基礎 ▁ に ▁ あり ▁ 、 ▁ 感 性 ▁ 的 ▁ な ▁ 現象 ▁ を ▁ 現象 ▁ と ▁ し ▁ て ▁ 成立 ▁ さ ▁ せる ▁ 存在 論 ▁ 的 ▁ な ▁ 概念 ▁ と ▁ し ▁ て ▁ 主 客 ▁二 ▁ 元 ▁ 論 ▁ の ▁ 近代 ▁ 哲学 的 ▁ 図 式 ▁ を ▁ 解 体 ▁ さ ▁ せる ▁ もの ▁ と ▁ なる ▁ こと ▁ を ▁ メ ル ロ ▁= ▁ ポン ティ ▁ は ▁ 目 論 ん ▁ で ▁ い ▁ た ▁ 。 ▁『 ▁ 眼 ▁ と ▁ 精神 ▁ 』 ▁( ▁1961 ▁ 年 ▁) ▁ は ▁この ▁「 ▁ 肉 ▁ 」 ▁ の ▁ 概念 ▁ を ▁ 絵 画 ▁ や ▁ 彫 刻 ▁ など ▁ の ▁ 美術 ▁ 作品 ▁ に 

In [8]:
# To save model and data
FileName = str(seednumber) + "_" + config.tree_infer_mode + '_Batch_' + str(batch_size) + 'Hidden_' + str(hidden_size) + \
            'LR' + str(lr) + "_" + str(time.time())

if not os.path.isdir(save_path):
    os.makedirs(save_path)
print("Model save path", save_path)
print("Finetune from path", finetune_frompath)
        

""" relation number is set at 42 """
if "en-rstdt" in data_path:
    number_of_relations = 42
elif "-gum" in data_path or "-gcdt" in data_path:
    number_of_relations = 30
model = ParsingNet(language_model, hidden_size, hidden_size,
                    hidden_size, atten_model, classifier_input_size, classifier_hidden_size, number_of_relations,
                    classifier_bias, rnn_layers, dropout_e, dropout_d, dropout_c, bert_tokenizer=bert_tokenizer)

if USE_CUDA:
    model = model.cuda()


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

    

Model save path depth_mode/Savings\jp-gcdt-xlm-roberta-base_bs1_seed111\\
Finetune from path None


In [9]:
with torch.no_grad():  # reduce memory
        # Load model
        torchsave_files = sorted(glob.glob(save_path + "Epoch_*.torchsave"))
        assert len(torchsave_files) == 1
        torchsave_best = torchsave_files[0]
        best_epoch_Dev = int(re.search(r"\d+", os.path.basename(torchsave_best)).group())
            
        model.load_state_dict(torch.load(torchsave_best, map_location=device))

        # Convert model to eval
        model.eval()

        # Eval on Testing data
        LossTree_Test, LossLabel_Test, span_points_Test, relation_points_Test, \
        nuclearity_points_Test, F1_full_Test, segment_points_Test = getAccuracy(Test_InputSentences,
                                                                                Test_EDUBreaks,
                                                                                Test_DecoderInput,
                                                                                Test_RelationLabel,
                                                                                Test_ParsingBreaks,
                                                                                Test_GoldenMetric,
                                                                                use_pred_segmentation=False,
                                                                                use_org_Parseval=use_org_Parseval,
                                                                                batch_size=eval_size,
                                                                                model=model)
        
        # Unfold numbers
        # Test
        P_span_Test, R_span_Test, F_span_Test = span_points_Test
        P_relation_Test, R_relation_Test, F_relation_Test = relation_points_Test
        P_nuclearity_Test, R_nuclearity_Test, F_nuclearity_Test = nuclearity_points_Test
        P_segment_Test, R_segment_Test, F_segment_Test = segment_points_Test
        
        print('Epoch EvalOnly Test:\t%d\n' % best_epoch_Dev,
              'F_segment_Test\tF_span_Test\tF_nuclearity_Test\tF_relation_Test:\n%.4f\t%.4f\t%.4f\t%.4f'
              % (F_segment_Test, F_span_Test, F_nuclearity_Test, F_relation_Test))
        
        # Eval on each document
        for docid in range(len(Test_InputSentences)):
            LossTree_Test, LossLabel_Test, span_points_Test, relation_points_Test, \
            nuclearity_points_Test, F1_full_Test, segment_points_Test = getAccuracy([Test_InputSentences[docid]],
                                                                                    [Test_EDUBreaks[docid]],
                                                                                    [Test_DecoderInput[docid]],
                                                                                    [Test_RelationLabel[docid]],
                                                                                    [Test_ParsingBreaks[docid]],
                                                                                    [Test_GoldenMetric[docid]],
                                                                                    use_pred_segmentation=False,
                                                                                    use_org_Parseval=use_org_Parseval,
                                                                                    batch_size=eval_size,
                                                                                    model=model)
            
            # Unfold numbers
            # Test
            P_span_Test, R_span_Test, F_span_Test = span_points_Test
            P_relation_Test, R_relation_Test, F_relation_Test = relation_points_Test
            P_nuclearity_Test, R_nuclearity_Test, F_nuclearity_Test = nuclearity_points_Test
            P_segment_Test, R_segment_Test, F_segment_Test = segment_points_Test
            
            print('Epoch EvalOnly Test:\t%d\tdocid:\t%d\n' % (best_epoch_Dev, docid),
                  'F_segment_Test\tF_span_Test\tF_nuclearity_Test\tF_relation_Test:\n%.4f\t%.4f\t%.4f\t%.4f'
                  % (F_segment_Test, F_span_Test, F_nuclearity_Test, F_relation_Test))

Epoch EvalOnly Test:	26
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7250	0.5203	0.4758
Epoch EvalOnly Test:	26	docid:	0
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7158	0.5579	0.4737
Epoch EvalOnly Test:	26	docid:	1
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7500	0.6062	0.5500
Epoch EvalOnly Test:	26	docid:	2
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.6204	0.4537	0.3981
Epoch EvalOnly Test:	26	docid:	3
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7943	0.5455	0.4833
Epoch EvalOnly Test:	26	docid:	4
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7137	0.4824	0.4706
Epoch EvalOnly Test:	26	docid:	5
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7065	0.4891	0.4565


In [10]:
LoopNeeded = int(np.ceil(len(Test_EDUBreaks) / batch_size))

Loss_tree_all = []
Loss_label_all = []
correct_span = 0
correct_relation = 0
correct_nuclearity = 0
correct_full = 0
no_system = 0
no_golden = 0
no_gold_seg = 0
no_pred_seg = 0
no_correct_seg = 0

correct_span_list = []
correct_relation_list = []
correct_nuclearity_list = []
no_system_list = []
no_golden_list = []

all_label_gold = []
all_label_pred = []

In [11]:
from Training import getBatchData

with torch.no_grad():
    for loop in range(LoopNeeded):
        
        StartPosition = loop * batch_size
        EndPosition = (loop + 1) * batch_size
        if EndPosition > len(Test_EDUBreaks):
            EndPosition = len(Test_EDUBreaks)

        InputSentences_batch, EDUBreaks_batch, _, RelationLabel_batch, ParsingBreaks_batch, GoldenMetric_batch = \
            getBatchData(Test_InputSentences[StartPosition:EndPosition],
                         Test_EDUBreaks[StartPosition:EndPosition],
                         Test_DecoderInput[StartPosition:EndPosition],
                         Test_RelationLabel[StartPosition:EndPosition],
                         Test_ParsingBreaks[StartPosition:EndPosition],
                         Test_GoldenMetric[StartPosition:EndPosition], batch_size)

        Loss_tree_batch, Loss_label_batch, SPAN_batch, Label_Tuple_batch, predict_EDU_breaks = model.TestingLoss(
            InputSentences_batch, EDUBreaks_batch, RelationLabel_batch,
            ParsingBreaks_batch, GenerateTree=True, use_pred_segmentation=False)

        all_label_gold.extend(Label_Tuple_batch[0])
        all_label_pred.extend(Label_Tuple_batch[1])

        Loss_tree_all.append(Loss_tree_batch)
        Loss_label_all.append(Loss_label_batch)

        correct_span_batch, correct_relation_batch, correct_nuclearity_batch, correct_full_batch, no_system_batch, no_golden_batch, \
        correct_span_batch_list, correct_relation_batch_list, correct_nuclearity_batch_list, \
        no_system_batch_list, no_golden_batch_list, segment_results_list = getBatchMeasure(SPAN_batch,
                                                                                           GoldenMetric_batch,
                                                                                           predict_EDU_breaks,
                                                                                           EDUBreaks_batch,
                                                                                           use_org_Parseval)

        correct_span = correct_span + correct_span_batch
        correct_relation = correct_relation + correct_relation_batch
        correct_nuclearity = correct_nuclearity + correct_nuclearity_batch
        correct_full = correct_full + correct_full_batch
        no_system = no_system + no_system_batch
        no_golden = no_golden + no_golden_batch
        no_gold_seg += segment_results_list[0]
        no_pred_seg += segment_results_list[1]
        no_correct_seg += segment_results_list[2]

        correct_span_list += correct_span_batch_list
        correct_relation_list += correct_relation_batch_list
        correct_nuclearity_list += correct_nuclearity_batch_list
        no_system_list += no_system_batch_list
        no_golden_list += no_golden_batch_list

In [12]:
span_points, relation_points, nuclearity_points, F1_Full, segment_points = getMicroMeasure(correct_span,
                                                                                           correct_relation,
                                                                                           correct_nuclearity,
                                                                                           correct_full,
                                                                                           no_system,
                                                                                           no_golden,
                                                                                           no_gold_seg,
                                                                                           no_pred_seg,
                                                                                           no_correct_seg)

In [13]:
with torch.no_grad():
    Loss_tree, Loss_label, SPAN, Label_Tuple, predict_EDU_breaks = model.TestingLoss(
        Test_InputSentences, Test_EDUBreaks, Test_RelationLabel,
        Test_ParsingBreaks, GenerateTree=True, use_pred_segmentation=False)

In [14]:
correct_span_1, correct_relation_1, correct_nuclearity_1, correct_full_1, no_system_1, no_golden_1, \
        correct_span_list_1, correct_relation_list_1, correct_nuclearity_list_1, \
        no_system_list_1, no_golden_list_1, segment_results_list_1 = getBatchMeasure(SPAN,
                                                                                     Test_GoldenMetric,
                                                                                     predict_EDU_breaks,
                                                                                     Test_EDUBreaks,
                                                                                     use_org_Parseval)

In [15]:
predict_dic_list = []
for i in range(len(SPAN)):
    predict_dic_list.append(Metric.getEvalData_parseval(SPAN[i][0], predict_EDU_breaks[i]))

In [16]:
gold_dic_list = []
for i in range(len(Test_GoldenMetric)):
    gold_dic_list.append(Metric.getEvalData_parseval(Test_GoldenMetric[i][0], Test_EDUBreaks[i]))

In [17]:
predict_dic_list_jp = predict_dic_list[:3]
predict_dic_list_gcdt = predict_dic_list[3:]
gold_dic_list_jp = gold_dic_list[:3]
gold_dic_list_gcdt = gold_dic_list[3:]

# Report on Accuracy

In [18]:
def get_accuracy(predict_dic_list, gold_dic_list):
    
    span_correct = 0
    nuc_correct = 0
    rel_correct = 0
    length = 0
    
    for i in range(len(gold_dic_list)):
        length += len(gold_dic_list[i])
        for key in gold_dic_list[i].keys():
            if key in predict_dic_list[i].keys():
                span_correct += 1
                if gold_dic_list[i][key][0] == predict_dic_list[i][key][0]:
                    rel_correct += 1
                if gold_dic_list[i][key][1] == predict_dic_list[i][key][1]:
                    nuc_correct += 1
    
    span = span_correct / length
    nuc = nuc_correct / length
    rel = rel_correct / length
    
    return span, nuc, rel

In [19]:
span, nuc, rel = get_accuracy(predict_dic_list_jp, gold_dic_list_jp)
print(span)
print(nuc)
print(rel)

0.7024793388429752
0.5482093663911846
0.48484848484848486


In [20]:
span, nuc, rel = get_accuracy(predict_dic_list_gcdt, gold_dic_list_gcdt)
print(span)
print(nuc)
print(rel)

0.7376543209876543
0.5046296296296297
0.470679012345679


# Report on Rel Types

In [21]:
def report_rel_type(predict_dic_list, gold_dic_list):
    
    relname_accuracy = {}
    
    for i in range(len(gold_dic_list)):
        for key in gold_dic_list[i].keys():
            rel = gold_dic_list[i][key][0]
            nuc = gold_dic_list[i][key][1]
            if rel not in relname_accuracy:
                relname_accuracy[rel] = [0, 1]
            else:
                relname_accuracy[rel][1] += 1
            if key in predict_dic_list[i].keys():
                if rel == predict_dic_list[i][key][0] and nuc == predict_dic_list[i][key][1]:
                    relname_accuracy[rel][0] +=1
    
    return relname_accuracy

In [22]:
relname_accuracy_jp = report_rel_type(predict_dic_list_jp, gold_dic_list_jp)
relname_accuracy_gcdt = report_rel_type(predict_dic_list_gcdt, gold_dic_list_gcdt)

In [23]:
relname_accuracy_jp

{'organization': [4, 6],
 'elaboration': [40, 76],
 'attribution': [4, 6],
 'joint': [50, 112],
 'context': [2, 27],
 'same-unit': [32, 45],
 'adversative': [7, 14],
 'contingency': [3, 7],
 'purpose': [1, 4],
 'explanation': [22, 30],
 'restatement': [0, 1],
 'mode': [5, 10],
 'causal': [3, 20],
 'evaluation': [0, 4],
 'topic': [0, 1]}

In [24]:
relname_accuracy_gcdt

{'organization': [16, 39],
 'explanation': [11, 41],
 'joint': [92, 193],
 'elaboration': [60, 99],
 'context': [12, 42],
 'same-unit': [52, 67],
 'mode': [3, 11],
 'attribution': [22, 35],
 'causal': [6, 47],
 'evaluation': [0, 10],
 'adversative': [10, 36],
 'purpose': [3, 17],
 'restatement': [1, 6],
 'contingency': [4, 4],
 'topic': [0, 1]}

# Report on Intra- and Inter-sentential Performances

In [25]:
sent_seg = []
for i in range(len(Test_EDUBreaks)):
    doc_sent_seg = []
    for index in Test_EDUBreaks[i]:
        if Test_InputSentences[i][index] in ['。', '？', '！', '.', '?', '!']:
            doc_sent_seg.append(index)
    sent_seg.append(doc_sent_seg)

In [26]:
sent_seg_jp = sent_seg[:3]
sent_seg_gcdt = sent_seg[3:]

In [27]:
def filter_intra(dic_list, sent_seg):
    
    filtered = []
    
    for i in range(len(dic_list)):
        
        result = {}
        
        for key in dic_list[i].keys():
            start, end = map(int, key.split('-'))
            count_in_range = sum(start <= num < end for num in sent_seg[i])
            if count_in_range == 0:
                result.update({key: dic_list[i][key]})
                
        filtered.append(result)
    
    return filtered

In [28]:
intra_gold_jp = filter_intra(gold_dic_list_jp, sent_seg_jp)
intra_gold_gcdt = filter_intra(gold_dic_list_gcdt, sent_seg_gcdt)

In [29]:
inter_gold_jp = []
inter_gold_gcdt = []

for i in range(len(gold_dic_list_jp)):
    inter_gold_jp.append({key : value for key, value in gold_dic_list_jp[i].items() if key not in intra_gold_jp[i]})
for i in range(len(gold_dic_list_gcdt)):
    inter_gold_gcdt.append({key : value for key, value in gold_dic_list_gcdt[i].items() if key not in intra_gold_gcdt[i]})

In [30]:
report_rel_type(predict_dic_list_jp, intra_gold_jp)

{'elaboration': [38, 54],
 'same-unit': [32, 45],
 'joint': [32, 53],
 'contingency': [3, 7],
 'purpose': [1, 4],
 'adversative': [7, 11],
 'mode': [5, 10],
 'explanation': [20, 21],
 'context': [0, 8],
 'causal': [3, 12],
 'attribution': [3, 4]}

In [31]:
report_rel_type(predict_dic_list_gcdt, intra_gold_gcdt)

{'explanation': [3, 15],
 'organization': [1, 6],
 'context': [11, 24],
 'same-unit': [52, 67],
 'elaboration': [54, 76],
 'joint': [62, 102],
 'mode': [3, 11],
 'attribution': [16, 24],
 'causal': [6, 34],
 'evaluation': [0, 7],
 'adversative': [8, 25],
 'purpose': [3, 17],
 'restatement': [1, 5],
 'contingency': [4, 4],
 'topic': [0, 1]}

In [32]:
report_rel_type(predict_dic_list_jp, inter_gold_jp)

{'organization': [4, 6],
 'attribution': [1, 2],
 'joint': [18, 59],
 'context': [2, 19],
 'adversative': [0, 3],
 'elaboration': [2, 22],
 'explanation': [2, 9],
 'restatement': [0, 1],
 'causal': [0, 8],
 'evaluation': [0, 4],
 'topic': [0, 1]}

In [33]:
report_rel_type(predict_dic_list_gcdt, inter_gold_gcdt)

{'organization': [15, 33],
 'joint': [30, 91],
 'explanation': [8, 26],
 'elaboration': [6, 23],
 'context': [1, 18],
 'causal': [0, 13],
 'adversative': [2, 11],
 'evaluation': [0, 3],
 'attribution': [6, 11],
 'restatement': [0, 1]}

In [34]:
span, nuc, rel = get_accuracy(predict_dic_list_jp, intra_gold_jp)
print(span)
print(nuc)
print(rel)

0.868995633187773
0.7292576419213974
0.6419213973799127


In [35]:
span, nuc, rel = get_accuracy(predict_dic_list_gcdt, intra_gold_gcdt)
print(span)
print(nuc)
print(rel)

0.8301435406698564
0.6028708133971292
0.5645933014354066


In [36]:
span, nuc, rel = get_accuracy(predict_dic_list_jp, inter_gold_jp)
print(span)
print(nuc)
print(rel)

0.417910447761194
0.23880597014925373
0.21641791044776118


In [37]:
span, nuc, rel = get_accuracy(predict_dic_list_gcdt, inter_gold_gcdt)
print(span)
print(nuc)
print(rel)

0.5695652173913044
0.32608695652173914
0.3


In [38]:
print(sum(len(doc) for doc in intra_gold_jp) / sum(len(doc) for doc in gold_dic_list_jp))
print(sum(len(doc) for doc in intra_gold_gcdt) / sum(len(doc) for doc in gold_dic_list_gcdt))

0.6308539944903582
0.6450617283950617


# Report on Intra-sentential Spans in the Whole Dataset

In [39]:
EDUBreaks = []
GoldenMetric = []
InputSentences = []
EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "EDUBreaks.pickle"), "rb")))
GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "GoldenLabelforMetric.pickle"), "rb")))
InputSentences.extend(pickle.load(open(os.path.join(data_path, "InputSentences.pickle"), "rb")))

In [40]:
seg = []
for i in range(len(EDUBreaks)):
    doc_sent_seg = []
    for index in EDUBreaks[i]:
        if InputSentences[i][index] in ['。', '？', '！', '.', '?', '!']:
            doc_sent_seg.append(index)
    seg.append(doc_sent_seg)

In [41]:
seg_jp_all = seg[:30]
seg_gcdt_all = seg[30:]

In [42]:
gold_dic_list_all = []
for i in range(len(GoldenMetric)):
    gold_dic_list_all.append(Metric.getEvalData_parseval(GoldenMetric[i][0], EDUBreaks[i]))

In [43]:
gold_jp_all = gold_dic_list_all[:30]
gold_gcdt_all = gold_dic_list_all[30:]

In [44]:
intra_jp_all = filter_intra(gold_jp_all, seg_jp_all)
intra_gcdt_all = filter_intra(gold_gcdt_all, seg_gcdt_all)

In [45]:
print(sum(len(doc) for doc in intra_jp_all) / sum(len(doc) for doc in gold_jp_all))
print(sum(len(doc) for doc in intra_gcdt_all) / sum(len(doc) for doc in gold_gcdt_all))

0.6886212828704842
0.6997885835095138


In [46]:
from collections import defaultdict
dic_jp = defaultdict(int)
dic_gcdt = defaultdict(int)

In [47]:
for doc in intra_gcdt_all:
    for key in doc.keys():
        dic_gcdt[doc[key][0]] += 1

In [48]:
dic_gcdt

defaultdict(int,
            {'causal': 175,
             'context': 247,
             'joint': 867,
             'explanation': 221,
             'same-unit': 729,
             'elaboration': 708,
             'purpose': 112,
             'restatement': 115,
             'mode': 112,
             'adversative': 181,
             'attribution': 315,
             'contingency': 60,
             'organization': 87,
             'evaluation': 42,
             'topic': 1})

In [51]:
for key in dic_gcdt.keys():
    dic_gcdt[key] = dic_gcdt[key] / 3972

In [52]:
dic_gcdt

defaultdict(int,
            {'causal': 0.04405840886203424,
             'context': 0.062185297079556896,
             'joint': 0.21827794561933533,
             'explanation': 0.05563947633434038,
             'same-unit': 0.18353474320241692,
             'elaboration': 0.1782477341389728,
             'purpose': 0.028197381671701913,
             'restatement': 0.028952668680765358,
             'mode': 0.028197381671701913,
             'adversative': 0.045568982880161125,
             'attribution': 0.07930513595166164,
             'contingency': 0.015105740181268883,
             'organization': 0.02190332326283988,
             'evaluation': 0.010574018126888218,
             'topic': 0.00025176233635448137})